In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("./wiki_data/wiki_graph_data.csv")
data

,Unnamed: 0,property,wikidata_id_start,wikidata_id_end,property_name,wikidata_id_start_name,wikidata_id_end_name
0,0,P414,Q4636301,Q13677,stock exchange,3D Systems,New York Stock Exchange
1,1,P31,Q4636301,Q4830453,instance of,3D Systems,business
2,2,P31,Q4636301,Q891723,instance of,3D Systems,public company
3,3,P159,Q4636301,Q960774,headquarters location,3D Systems,Rock Hill
4,4,P17,Q4636301,Q30,country,3D Systems,United States of America
...,...,...,...,...,...,...,...
26957,26957,P1830,Q230426,Q42766132,owner of,Zurich Insurance Group,"Calle de Alcalá 44, Madrid"
26958,26958,P452,Q230426,Q43183,industry,Zurich Insurance Group,insurance
26959,26959,P1056,Q230426,Q837171,product or material produced,Zurich Insurance Group,financial services
26960,26960,P361,Q230426,Q666714,part of,Zurich Insurance Group,Swiss Market Index


In [55]:
len(list(set(list(set(data.wikidata_id_start))+list(set(data.wikidata_id_end)))))

13274

In [3]:
# set(list(set(data.wikidata_id_end) - set(data.wikidata_id_start)))

In [15]:
# import requests 
# import pandas as pd
# def get_relations(wikidata_id):
#     # wikidata_id = "Q8073337"
#     try:
#         req = requests.get('https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json', params={'ids': wikidata_id})
#         a_dict = pd.DataFrame(req.json()["entities"]).T.claims.values[0]

#         relations = []
#         for key in a_dict:
#             for element in a_dict[key]:
#                 temp_dict = flatten_data(element)
#                 data_dict = {}
#                 if temp_dict["mainsnak_datatype"] == 'wikibase-item':
#                     data_dict["property"] = temp_dict["mainsnak_property"]
#                     data_dict["wikidata_id_start"] = wikidata_id
#                     data_dict["wikidata_id_end"] = temp_dict["mainsnak_datavalue_value_id"]
#                     relations.append(data_dict)
#         return relations
#     except:
#         return None
       

In [31]:
import grequests
import requests
class Test(object):
    def __init__(self, urls):
        self.urls = urls

    def exception(self, request, exception):
        print("Problem: {}: {}".format(request.url, exception))

    def async(self):
        results = grequests.map((grequests.get(u) for u in self.urls), exception_handler=self.exception, size=200)
        # print(results)
        return results

In [14]:
from tqdm import tqdm
from tqdm.contrib import tzip
# tqdm.pandas()
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out


queries = list(set(list(set(data.wikidata_id_end) - set(data.wikidata_id_start))))
urls_list = []
for x in queries:
    urls_list.append('https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}'.format(x))
    
relations = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 20))):
    urls = urls_list[i.start:i.stop]
    names = queries[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req, name in zip(results, names):
        if req != None:
            if req.status_code == 200:
                a_dict = pd.DataFrame(req.json()["entities"]).T.claims.values[0]
                for key in a_dict:
                    for element in a_dict[key]:
                        temp_dict = flatten_data(element)
                        
                        data_dict = {}
                        if temp_dict["mainsnak_datatype"] == 'wikibase-item':
                            data_dict["property"] = temp_dict["mainsnak_property"]
                            data_dict["wikidata_id_start"] = name
                            try: # not every wikibase-item has mainsnak_datavalue_value_id
                                data_dict["wikidata_id_end"] = temp_dict["mainsnak_datavalue_value_id"]
                                if data_dict not in relations:
                                    relations.append(data_dict)
                            except:
                                # print(temp_dict)
                                continue

            
    

100%|████████████████████████████████████████████████████████████████████████████████| 579/579 [24:56<00:00,  2.58s/it]


In [15]:
relations

[{'property': 'P31',
  'wikidata_id_start': 'Q67012747',
  'wikidata_id_end': 'Q4830453'},
 {'property': 'P1366',
  'wikidata_id_start': 'Q67012747',
  'wikidata_id_end': 'Q1545076'},
 {'property': 'P31',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q891723'},
 {'property': 'P112',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q197688'},
 {'property': 'P159',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q42651'},
 {'property': 'P1454',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q891723'},
 {'property': 'P355',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q30'},
 {'property': 'P452',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q192127'},
 {'property': 'P1056',
  'wikidata_id_start': 'Q1188943',
  'wikidata_id_end': 'Q58803'},
 {'property': 'P279',
  'wikidata_id_start': 'Q4503',
  'wikidata_id_end': 'Q1546066'},
 {'property': 'P279',
  'wikidata_id_start': 'Q4503',
  'wikidata_id_end': 'Q193446'},
 {'property': 'P

In [16]:
pd.DataFrame(relations)

,property,wikidata_id_start,wikidata_id_end
0,P31,Q67012747,Q4830453
1,P1366,Q67012747,Q1545076
2,P31,Q1188943,Q891723
3,P112,Q1188943,Q197688
4,P159,Q1188943,Q42651
...,...,...,...
142845,P21,Q11578412,Q6581097
142846,P106,Q11578412,Q43845
142847,P27,Q11578412,Q17
142848,P69,Q11578412,Q204626


c:\users\qhuca\anaconda3\envs\download\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [19]:
data2 = pd.DataFrame(relations)

In [25]:
df = pd.concat([data2[["property","wikidata_id_start","wikidata_id_end"]], data[["property","wikidata_id_start","wikidata_id_end"]]])

In [26]:
# df.to_csv("wiki_graph_data_2hop.csv")

In [35]:
# get property name for wikidata relations
# from bs4 import BeautifulSoup
# "https://www.wikidata.org/w/api.php?action=wbgetentities&ids=P31&languages=en"
# tqdm.pandas()
def get_property_name(property_id):
    url = 'https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}'.format(property_id)
    req = requests.get(url)
    return req.json()["entities"][property_id]['labels']["en"]["value"]

queries = df["property"].values.tolist()
urls_list = []
for x in queries:
    urls_list.append('https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}'.format(x))
    
prop_names = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 200))):
    urls = urls_list[i.start:i.stop]
    names = queries[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req, name in zip(results, names):
        if req != None:
            if req.status_code == 200:
                prop_names.append(req.json()["entities"][name]['labels']["en"]["value"])
                                
df["property_name"] = prop_names

100%|████████████████████████████████████████████████████████████████████████████████| 850/850 [38:12<00:00,  2.70s/it]


In [39]:
# # get name for wikidata id
# def get_wiki_id_name(wiki_id):
#     try:
#         url = 'https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}&props=labels'.format(wiki_id)
#         req = requests.get(url)
#         return req.json()["entities"][wiki_id]["labels"]["en"]["value"]
#     except:
#         return None

# relations_df["wikidata_id_start_name"] = df["wikidata_id_start"].progress_apply(get_wiki_id_name)
# relations_df["wikidata_id_end_name"] = relations_df["wikidata_id_end"].progress_apply(get_wiki_id_name)
queries = df["wikidata_id_start"].values.tolist()
urls_list = []
for x in queries:
    urls_list.append('https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}&props=labels'.format(x))
    
start_names = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 200))):
    urls = urls_list[i.start:i.stop]
    names = queries[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req, name in zip(results, names):
        if req != None:
            if req.status_code == 200:
                try:
                    start_names.append(req.json()["entities"][name]["labels"]["en"]["value"])
                except:
                    start_names.append(None)
                                
df["wikidata_id_start_name"] = start_names
########################################
queries = df["wikidata_id_end"].values.tolist()
urls_list = []
for x in queries:
    urls_list.append('https://www.wikidata.org/w/api.php?action=wbgetentities&languages=en&format=json&ids={}&props=labels'.format(x))
    
end_names = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 200))):
    urls = urls_list[i.start:i.stop]
    names = queries[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req, name in zip(results, names):
        if req != None:
            if req.status_code == 200:
                try:
                    end_names.append(req.json()["entities"][name]["labels"]["en"]["value"])
                except:
                    end_names.append(None)
            
                                
df["wikidata_id_end_name"] = end_names

100%|████████████████████████████████████████████████████████████████████████████████| 850/850 [32:27<00:00,  2.29s/it]


In [68]:


df_wiki = pd.concat([df[["wikidata_id_start", "wikidata_id_start_name"]].rename(columns={"wikidata_id_start": "wiki_id", "wikidata_id_start_name": "name"}), df[["wikidata_id_end", "wikidata_id_end_name"]].rename(columns={"wikidata_id_end": "wiki_id", "wikidata_id_end_name": "name"})])
df_wiki = df_wiki[["wiki_id", "name"]].drop_duplicates()


from urllib.parse import urlparse
from tqdm import tqdm
tqdm.pandas()

# def get_wikipedia_url_from_wikidata_id(wikidata_id, lang='en', debug=False):
#     import requests
#     from requests import utils

#     url = (
#         'https://www.wikidata.org/w/api.php'
#         '?action=wbgetentities'
#         '&props=sitelinks/urls'
#         f'&ids={wikidata_id}'
#         '&format=json')
#     json_response = requests.get(url).json()
#     if debug: print(wikidata_id, url, json_response) 

#     entities = json_response.get('entities')    
#     if entities:
#         entity = entities.get(wikidata_id)
#         if entity:
#             sitelinks = entity.get('sitelinks')
#             if sitelinks:
#                 if lang:
#                     # filter only the specified language
#                     sitelink = sitelinks.get(f'{lang}wiki')
#                     if sitelink:
#                         wiki_url = sitelink.get('url')
#                         if wiki_url:
#                             return requests.utils.unquote(wiki_url)
#                 else:
#                     # return all of the urls
#                     wiki_urls = {}
#                     for key, sitelink in sitelinks.items():
#                         wiki_url = sitelink.get('url')
#                         if wiki_url:
#                             wiki_urls[key] = requests.utils.unquote(wiki_url)
#                     return wiki_urls
#     return None   

# def get_url(wiki_id):
#     return get_wikipedia_url_from_wikidata_id(wiki_id, lang='en')
    
# df_wiki["url"] = df_wiki["wiki_id"].progress_apply(get_url)



queries = df_wiki["wiki_id"].values.tolist()
urls_list = []
for x in queries:
    urls_list.append('https://www.wikidata.org/w/api.php?action=wbgetentities&props=sitelinks/urls&ids={}&format=json'.format(x))
    
wiki_urls = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 200))):
    urls = urls_list[i.start:i.stop]
    names = queries[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req, name in zip(results, names):
        if req != None:
            if req.status_code == 200:
                try:
                    wiki_urls.append(req.json()["entities"][name]["sitelinks"]["enwiki"]["url"])
                except:
                    wiki_urls.append(None)
            

df_wiki["url"] = wiki_urls


100%|████████████████████████████████████████████████████████████████████████████████| 353/353 [13:12<00:00,  2.25s/it]
c:\users\qhuca\anaconda3\envs\download\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [69]:
df.to_csv("./new/wiki_graph_data_2hop.csv")

In [70]:
def process_url(url):
    if url != None:
        p = urlparse(url)
        return "https://"+p.netloc+"/w/index.php?action=raw&title="+p.path.split("/")[-1]
    else:
        return None

df_wiki["raw_url"] = df_wiki["url"].progress_apply(process_url)
df_wiki = df_wiki.dropna()

import requests 
from bs4 import BeautifulSoup
# def extract_text_from_url(url):
#     try:
#         # url = 'https://www.mopo.de/shoppingwelt/raclette-grill-fonduelette-mini-pizzaofen-33588944'
#         req = requests.get(url, timeout=10)
#         soup = BeautifulSoup(req.content, 'html.parser')
#         return " ".join(soup.findAll(text=True))
#     except:
#         return None


urls_list = df_wiki["raw_url"].values.tolist()
descriptions = []
for i in tqdm(list(chunks(range(0, len(urls_list)), 200))):
    urls = urls_list[i.start:i.stop]
    test = Test(urls)
    results = test.async()
    for req in results:
        if req != None:
            if req.status_code == 200:
                soup = BeautifulSoup(req.content, 'html.parser')
                descriptions.append(" ".join(soup.findAll(text=True)))
            else:
                descriptions.append(None)
        else:
            descriptions.append(None)

df_wiki["descriptions"] = descriptions

100%|████████████████████████████████████████████████████████████████████████████████| 249/249 [54:02<00:00, 13.02s/it]
c:\users\qhuca\anaconda3\envs\download\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [73]:
df_wiki.dropna()

,wiki_id,name,url,raw_url,descriptions
2,Q1188943,Suntech Power,https://en.wikipedia.org/wiki/Suntech_Power,https://en.wikipedia.org/w/index.php?action=ra...,{{Infobox company\n| name = Suntec...
9,Q4503,network switch,https://en.wikipedia.org/wiki/Network_switch,https://en.wikipedia.org/w/index.php?action=ra...,{{Short description|Networking hardware that f...
26,Q836827,Celcom,https://en.wikipedia.org/wiki/Celcom,https://en.wikipedia.org/w/index.php?action=ra...,{{Distinguish|Cellcom (disambiguation){{!}}Cel...
34,Q26943,Trelleborg,https://en.wikipedia.org/wiki/Trelleborg,https://en.wikipedia.org/w/index.php?action=ra...,"{{short description|town in Skåne, Sweden}}\n{..."
46,Q6926887,Moviefone,https://en.wikipedia.org/wiki/Moviefone,https://en.wikipedia.org/w/index.php?action=ra...,{{short description|Movie listing service}}\n{...
...,...,...,...,...,...
18206,Q1131760,compactor,https://en.wikipedia.org/wiki/Compactor,https://en.wikipedia.org/w/index.php?action=ra...,{{refimprove|date=February 2018}}\n[[Image:Cat...
19570,Q935396,getter,https://en.wikipedia.org/wiki/Getter,https://en.wikipedia.org/w/index.php?action=ra...,{{Short description|Consumable reactive substa...
19921,Q16908988,Saudi Refining,https://en.wikipedia.org/wiki/Saudi_Refining,https://en.wikipedia.org/w/index.php?action=ra...,#REDIRECT [[Saudi Aramco]]
24180,Q24697037,Unilever Bangladesh Limited,https://en.wikipedia.org/wiki/Unilever_Banglad...,https://en.wikipedia.org/w/index.php?action=ra...,#REDIRECT [[Unilever]]


c:\users\qhuca\anaconda3\envs\download\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [76]:
with open("./new/wiki_graph_data_2hop_description.csv", "w", encoding="utf-8_sig", errors='ignore') as f:
    df_wiki.to_csv(f)

In [ ]:
#end